# Making Pipeline for selected model
We have selected `xgboostregressor` model for our app. Accourding to `RMSE` AdaboostRegressor was little better than xgboost. but my system RAM was getting __crash while training__ and converting into pickle file. Hence I decided to continue with `xgboost`

In [114]:
#load data
import pandas as pd
data = pd.read_csv('cleaned_data2.csv', index_col='Unnamed: 0')
data = data.drop('seller_type',axis=1)
data.head()

,name,selling_price,km_driven,fuel,transmission,owner,age,company
0,Maruti 800 AC,60000,70000,Petrol,Manual,First Owner,15,Maruti
1,Maruti Wagon R LXI Minor,135000,50000,Petrol,Manual,First Owner,15,Maruti
2,Hyundai Verna 1.6 SX,600000,100000,Diesel,Manual,First Owner,10,Hyundai
3,Datsun RediGO T Option,250000,46000,Petrol,Manual,First Owner,5,Datsun
4,Honda Amaze VX i-DTEC,450000,141000,Diesel,Manual,Second Owner,8,Honda


#### Import libraries

In [99]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from xgboost import XGBRegressor
import pickle

#### fitting Onehotencoding 

In [102]:
ohe = OneHotEncoder().fit(data[['fuel','transmission','company']])
ohe.categories_

[array(['CNG', 'Diesel', 'LPG', 'Petrol'], dtype=object),
 array(['Automatic', 'Manual'], dtype=object),
 array(['Ambassador', 'Ashok', 'Audi', 'BMW', 'Chevrolet', 'Daewoo',
        'Datsun', 'Fiat', 'Force', 'Ford', 'Honda', 'Hyundai', 'Isuzu',
        'Jaguar', 'Jeep', 'Kia', 'Land', 'MG', 'Mahindra', 'Maruti',
        'Mercedes', 'Mitsubishi', 'Nissan', 'OpelCorsa', 'Renault',
        'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'], dtype=object)]

## ColumnTransformer and Pipeline

In [103]:
transformer = ColumnTransformer(remainder='drop', transformers=[('encoder',OneHotEncoder(categories=ohe.categories_),['fuel','transmission','company']),
                                                                        ('scaler',StandardScaler(),['km_driven','age'])])

In [104]:
pipe = Pipeline(steps=[('trans',transformer),('xgboost',XGBRegressor())])

In [105]:
X = data.drop('selling_price',axis=1)
y = data[['selling_price']]

In [106]:
test1 = pipe.fit(X,y)

## Saving model as pkl

In [107]:
with open('xgboost_model.pkl','wb') as files:
    pickle.dump(pipe, files)

In [112]:
test = pd.DataFrame([['Maruti 800 AC','2700','Petrol','Manual','First Owner',10,'Maruti']],columns=['name', 'km_driven', 'fuel', 'transmission', 'owner', 'age','company'])

In [113]:
pipe.predict(test)

array([219656.84], dtype=float32)